# Experiment 2. Preliminary data import for the main experiment.

In [1]:
import os
import pandas as pd
import numpy as np
import os

In [2]:
### constants
data_folder= 'Experiment 2'
buttons= ['J', 'K']
event_code= ['Pressed', 'Released']
observerN= 15

In [10]:
### block conditions for each observers
observer_conditions= pd.read_csv(os.path.join(data_folder, 'Experiment2_BlockList.csv'), sep=';')

### Importing all responses into a single table `Experiment2_Response.csv`

In [24]:
### importing all responses into a single table
dfDict= {'ID': [], 'Block':[], 'ConditionLabel': [], 'Time': [], 'Key': [], 'EventLabel': [], 'Target': [], 'Event': []}
for iO in range(1, observerN+1):
    with open(os.path.join(data_folder, 's%02d.txt'%(iO)), 'rt') as response_file:
        current_block= 1
        block_has_started= False
        data_lines= 0
        for current_line in response_file:
            # skipping the "start of the block" line
            if current_line[:len("--Activated")]=="--Activated":
                # advancing block counter
                if block_has_started:
                    ## end of the previous block
                    dfDict['ID'].append('s%02d'%(iO))
                    dfDict['Block'].append(current_block)
                    dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
                    dfDict['Time'].append(possible_end_of_block_time)
                    dfDict['Key'].append(subfields[-1][0])
                    dfDict['EventLabel'].append('Block end')
                    dfDict['Target'].append(-1)
                    dfDict['Event'].append(200)
                    
                    # preparing counters for the next block
                    current_block+= 1
                    block_has_started= False
                    data_lines= 0
                continue
                
            # skipping over an empty line
            if len(current_line.strip())==0:
                continue
            
            subfields= current_line.split()
            # parsing time into seconds
            time_components= subfields[0].split(':') # assuming the hours:minutes:float_seconds
            if len(time_components)==1: # 
                time_components= subfields[0].split('-') # nope, this is hours-minutes-float_seconds
            if len(time_components)==1:
                continue # yet something else, definitely not the relevant response line
            four_time_components= np.concatenate([time_components[0:-1], time_components[-1].split('.')])
            time_in_seconds= np.sum(np.array(four_time_components).astype('float')*np.array([60*60, 60, 1, 1/100.0]))

            if subfields[-1][0] in buttons: # response buttons
                data_lines+= 1
                # adding the next line to the dictionary
                dfDict['ID'].append('s%02d'%(iO))
                dfDict['Block'].append(current_block)
                dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
                dfDict['Time'].append(time_in_seconds)

                # parsing event into target and event type
                dfDict['Key'].append(subfields[-1][0])
                dfDict['EventLabel'].append(subfields[-1].strip()[2:-1])
                dfDict['Target'].append(buttons.index(subfields[-1][0]))
                dfDict['Event'].append(event_code.index(subfields[-1].strip()[2:-1]))
            elif subfields[-1][0] in ['E', 'R']: # special event
                if not block_has_started and subfields[-1].strip()[2:-1]=='Released': 
                    ## block start
                    dfDict['ID'].append('s%02d'%(iO))
                    dfDict['Block'].append(current_block)
                    dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
                    dfDict['Time'].append(time_in_seconds)
                    dfDict['Key'].append(subfields[-1][0])
                    dfDict['EventLabel'].append('Block start')
                    dfDict['Target'].append(-1)
                    dfDict['Event'].append(100)
                    block_has_started= True
                elif subfields[-1].strip()[2:-1]=='Pressed':
                    possible_end_of_block_time= time_in_seconds
            else:
                data_lines+= 1
                
    ## end of the last block
    dfDict['ID'].append('s%02d'%(iO))
    dfDict['Block'].append(current_block)
    dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
    dfDict['Time'].append(possible_end_of_block_time)
    dfDict['Key'].append(subfields[-1][0])
    dfDict['EventLabel'].append('Block end')
    dfDict['Target'].append(-1)
    dfDict['Event'].append(200)

            
responses= pd.DataFrame(dfDict, columns=['ID', 'Block', 'ConditionLabel', 'Time', 'Target', 'Event', 'Key', 'EventLabel'])
responses.to_csv(os.path.join(data_folder, 'Experiment2_Response.csv'), sep=';', index= False)

### Converting key presses into the sequence of subjective states, saved in `Experiment2_SubjectiveStateChange.csv`

In [28]:
### turning responses into the sequence of subjective states'
subjective_state_change= pd.DataFrame(columns= ['ID', 'Block', 'BlockDuration', 'ConditionLabel', 'Trial', 'TargetCount', 'Time', 'Duration'])
for current_ID in responses.ID.unique():
    print('Obsrver %s'%(current_ID), end=': block ')
    for current_block in responses.Block[responses.ID==current_ID].unique():
        print('%d'%(current_block), end=' ')
        iBlockTrials= responses[(responses.ID==current_ID) & (responses.Block==current_block)].index
        
        # figuring out block start-end-duration
        current_block_start= responses.Time[(responses.ID==current_ID) & (responses.Block==current_block) & (responses.EventLabel=='Block start')].values[0]
        current_block_end= responses.Time[(responses.ID==current_ID) & (responses.Block==current_block) & (responses.EventLabel=='Block end')].values[0]
        current_block_duration=current_block_end-current_block_start 

        # block constants
        current_entry= {'ID': current_ID, 
                        'Block': current_block, 
                        'BlockDuration': current_block_duration,
                        'ConditionLabel': responses.ConditionLabel[iBlockTrials[0]]}        
        
        # adding an onset state - all visibile
        current_entry['Trial']= 0
        current_entry['TargetCount']= 2
        current_entry['Duration']= 0
        current_entry['Time']= 0
        subjective_state_change.loc[subjective_state_change.shape[0]] = pd.Series(current_entry)
        
        CurrentTargetCount= 2
        PrevioiusEventTime= 0
        # going through events
        for iTrial, iRow in enumerate(iBlockTrials[1:-1]): # first and last events are labels for start and end of the block
            current_entry['Trial']= iTrial+1
            if responses.Event[iRow]==0:
                CurrentTargetCount-= 1
            else:
                CurrentTargetCount+= 1
            current_entry['TargetCount']= CurrentTargetCount
            current_entry['Time']=  responses.Time[iRow]-current_block_start
            subjective_state_change.set_value(len(subjective_state_change)-1, 'Duration', current_entry['Time']-PrevioiusEventTime)
            if iTrial==len(iBlockTrials)-2: # the last trial
                current_entry['Duration']= current_block_duration-current_entry['Time']
            PrevioiusEventTime= current_entry['Time']
            subjective_state_change.loc[subjective_state_change.shape[0]] = pd.Series(current_entry)
    print('\r', end='')
subjective_state_change.to_csv(os.path.join('Experiment 2', 'Experiment2_SubjectiveStateChange.csv'), sep=';', index= False)